In [1]:
import xarray as xr
# Need to use xbatcher from: https://github.com/arbennett/xbatcher/tree/develop
import xbatcher as xb
import numpy as np

from glob import glob
from parflow.tools.io import read_pfb

# Assembling all of the file paths we will need

In [2]:
BASE_DIR = '/hydrodata/PFCLM/CONUS1_baseline/simulations'
YEAR = 2004

# Get Pressure files
pressure_files = sorted(glob(f'{BASE_DIR}/{YEAR}/raw_outputs/pressure/*.pfb'))
pressure_files = {
    't': pressure_files[0:-1],
    't+1': pressure_files[1:]
}

# Get parameter filesk
parameter_names = [
    'permeability', 'porosity', 'vgn_alpha', 'vgn_n', 'slope_x', 'slope_y'
]
parameter_files = {
    name: f'{BASE_DIR}/static/CONUS1_{name}.pfb' for name in parameter_names
}


# Get forcing files
all_forcings = glob(f'{BASE_DIR}/{YEAR}/WY{YEAR}/*.pfb')
varnames = set([f.split('/')[-1].split('.')[1] for f in all_forcings])

variable_forcings = {}
for v in varnames:
    variable_forcings[v] = sorted(glob(f'{BASE_DIR}/{YEAR}/WY{YEAR}/*.{v}.*pfb'))

# Some constants, these might need to be pulled from data directly or user configurable

In [3]:
X_EXTENT = 3342
Y_EXTENT = 1888
T_EXTENT = 8759 # 1 less because we are predicting t+1
Z_EXTENT = 5
PATCH_SIZE = 128
PATCH_OVERLAP = 32

# Dummy data will be used to pull indices for reading subsets of the data

In [4]:
dummy_data = xr.Dataset().assign_coords({
    'time': np.arange(T_EXTENT),
    'z': np.arange(Z_EXTENT),
    'y': np.arange(Y_EXTENT),
    'x': np.arange(X_EXTENT)
})

# The batch generator create all of the sets of indices we need to sample the full dataset

In [ ]:
bgen = xb.BatchGenerator(
    dummy_data,
    input_dims={'x': PATCH_SIZE, 'y': PATCH_SIZE, 'time': 1},
    input_overlap={'x': PATCH_OVERLAP, 'y': PATCH_OVERLAP},
    return_partial=True,
    shuffle=True,
)

# Now you can see this pulls samples from the dummy data
# Normally you would loop over this, but I can just do 
# this to grab the first sample
sample_indices = next(iter(bgen))

#  Pulling the source and target pressure fields

In [ ]:
# Pulling the indices we need
time_index = sample_indices['time'].values[0]
x_min, x_max = sample_indices['x'].values[[0, -1]]
y_min, y_max = sample_indices['y'].values[[0, -1]]

# Setting up the keys dictionary
pressure_keys = {
    'x': {'start': x_min, 'stop': x_max+1},
    'y': {'start': y_min, 'stop': y_max+1},
}

# Construct the state data:
file_to_read = pressure_files['t'][time_index]
state_data = read_pfb(file_to_read, keys=pressure_keys)

# Construct the target data:
file_to_read_target = pressure_files['t+1'][time_index]
target_data = read_pfb(file_to_read_target, keys=pressure_keys)

# Forcings and targets now have dims
# (layers, y, x)
print(state_data.shape, target_data.shape)

(5, 128, 128) (5, 128, 128)


# Pulling the forcing data and stacking it all together

In [8]:
# Calculate which forcing file to choose
forcing_file_starts = np.arange(1, T_EXTENT, 24)
next_forcing_file_index = np.where((forcing_file_starts - time_index) < 0)[0][-1]

# Calculate what timestep to select out of the forcing file
forcing_file_z_index = time_index - forcing_file_starts[next_forcing_file_index]

# Put it in the keys format that read_pfb uses
forcing_keys = {
    'x': {'start': x_min, 'stop': x_max+1},
    'y': {'start': y_min, 'stop': y_max+1},
    # z acts as timesteps for forcing variables
    'z': {'start': forcing_file_z_index, 'stop': forcing_file_z_index + 1},
}

# Construct the forcing data:
forcing_data = []
for var in variable_forcings.keys():
    file_to_read = variable_forcings[var][next_forcing_file_index]
    forcing_data.append(read_pfb(file_to_read, keys=forcing_keys))

# Now just concatenate together.
# this works because we still have
# the leftover time index at dimension 1
# End result is dims of (n_forcing_vars, y, x)
forcing_data = np.concatenate(forcing_data)
forcing_data.shape

(8, 128, 128)

# Pulling all the parameter data and stacking it all together

In [9]:
parameter_data = []
for p, files in parameter_files.items():
    parameter_data.append(read_pfb(files, keys=pressure_keys))

# Concatenate the parameter data together
# End result is a dims of (n_parameters, y, x)
parameter_data = np.concatenate(parameter_data, axis=0)
parameter_data.shape

(22, 128, 128)